In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
import numpy as np
import pandas as pd
movies = pd.read_csv("/content/drive/MyDrive/Movie_Recommendation_System/tmdb_5000_movies.csv")
credits = pd.read_csv("/content/drive/MyDrive/Movie_Recommendation_System/tmdb_5000_credits.csv")
# movies.head(1)
# credits.head(1)['cast'].values
movies = movies.merge(credits, on='title')
movies = movies[['movie_id', 'title', 'overview', 'genres', 'keywords', 'cast', 'crew']]
# movies.head()

In [4]:
movies.isnull().sum()

movie_id    0
title       0
overview    3
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [5]:
movies.dropna(inplace=True)

In [6]:
movies.duplicated().sum()

0

In [7]:
movies.iloc[0].genres

'[{"id": 28, "name": "Action"}, {"id": 12, "name": "Adventure"}, {"id": 14, "name": "Fantasy"}, {"id": 878, "name": "Science Fiction"}]'

In [8]:
import ast

def convert(obj):
    L = []
    for i in ast.literal_eval(obj):
      L.append(i['name'])
    return L


In [9]:
movies['genres'] = movies['genres'].apply(convert)

In [10]:
movies['keywords'] = movies['keywords'].apply(convert)

In [11]:
def convert_top3(obj):
    L = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            L.append(i['name'])
            counter+=1
        else:
            break
    return L


In [12]:
movies['cast'] = movies['cast'].apply(convert_top3)

In [13]:
# movies.head()

In [14]:
def fetch_director(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            L.append(i['name'])
            break
    return L


In [15]:
movies['crew'] = movies['crew'].apply(fetch_director)

In [16]:
movies['overview'] = movies['overview'].apply(lambda x:x.split())

In [17]:
# movies.head()

In [18]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])

In [19]:
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])


In [20]:
# movies.head()

In [21]:
movies['tags'] = movies['overview'] + movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [22]:
# movies.head()

In [23]:
newdf = movies[['movie_id', 'title', 'tags']]

In [24]:
newdf['tags'] = newdf['tags'].apply(lambda x:" ".join(x))

<ipython-input-24-f3a66a72fa2d>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['tags'] = newdf['tags'].apply(lambda x:" ".join(x))


In [25]:
# newdf.head()

In [26]:
newdf['tags'][0]

'In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron'

In [27]:
newdf['tags'] = newdf['tags'].apply(lambda x:x.lower())

<ipython-input-27-9175ddfa9833>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['tags'] = newdf['tags'].apply(lambda x:x.lower())


In [28]:
# newdf.head()

In [29]:
!pip install nltk

In [30]:
import nltk

In [31]:
from nltk.stem.porter import PorterStemmer
ps = PorterStemmer()

In [32]:
def stem(text):
    y = []
    for i in text.split():
        y.append(ps.stem(i)) # if ['love','loved','loving'] will return love i.e the root term
    return " ".join(y)

In [33]:
newdf['tags'] = newdf['tags'].apply(stem)

<ipython-input-33-53d0dca3d307>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  newdf['tags'] = newdf['tags'].apply(stem)


In [34]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features = 5000, stop_words = 'english')

In [35]:
vectors = cv.fit_transform(newdf['tags']).toarray()

In [36]:
vectors

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [37]:
cv.get_feature_names_out()

array(['000', '007', '10', ..., 'zone', 'zoo', 'zooeydeschanel'],
      dtype=object)

In [38]:
stem('In the 22nd century, a paraplegic Marine is dispatched to the moon Pandora on a unique mission, but becomes torn between following orders and protecting an alien civilization. Action Adventure Fantasy ScienceFiction cultureclash future spacewar spacecolony society spacetravel futuristic romance space alien tribe alienplanet cgi marine soldier battle loveaffair antiwar powerrelations mindandsoul 3d SamWorthington ZoeSaldana SigourneyWeaver JamesCameron')

'in the 22nd century, a parapleg marin is dispatch to the moon pandora on a uniqu mission, but becom torn between follow order and protect an alien civilization. action adventur fantasi sciencefict cultureclash futur spacewar spacecoloni societi spacetravel futurist romanc space alien tribe alienplanet cgi marin soldier battl loveaffair antiwar powerrel mindandsoul 3d samworthington zoesaldana sigourneyweav jamescameron'

In [39]:
from sklearn.metrics.pairwise import cosine_similarity

In [40]:
similarity = cosine_similarity(vectors)

In [41]:
sorted(list(enumerate(similarity[0])), reverse = True, key = lambda x:x[1])[1:6]

[(1216, 0.28676966733820225),
 (2409, 0.26901379342448517),
 (3730, 0.2605130246476754),
 (507, 0.255608593705383),
 (539, 0.25038669783359574)]

In [42]:
def recommend(movie):
    movie_index = newdf[newdf['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(distances)), reverse = True, key = lambda x:x[1])[1:6]

    for i in movies_list:
        print(newdf.iloc[i[0]].title)
        # print(i)

In [43]:
recommend('Batman Begins')

The Dark Knight
Batman
Batman
The Dark Knight Rises
10th & Wolf


In [44]:
newdf.iloc[539]

movie_id                                                 7450
title                                              Titan A.E.
tags        a young man find out that he hold the key to r...
Name: 539, dtype: object

In [45]:
import pickle

In [46]:
pickle.dump(newdf, open('movies.pkl', 'wb'))

In [47]:
newdf['title'].values

array(['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre',
       ..., 'Signed, Sealed, Delivered', 'Shanghai Calling',
       'My Date with Drew'], dtype=object)

In [48]:
pickle.dump(newdf.to_dict(), open('movie_dict.pkl', 'wb'))

In [49]:
# pickle.dump(newdf.to_dict(), open('movies_dict.pkl', 'wb'))

In [50]:
pickle.dump(similarity, open("similarity.pkl", 'wb'))